# Consigna
Sobre el dataset del proyecto final, realizar la creación e implementación de al menos uno de los algoritmos de clasificación (KNN, Random Forest, Regresión Logística).

# Random Forest

In [ ]:
# Carga de las librerías
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# Cargamos los datos
df = pd.read_csv('/content/FIFA22_official_data.csv')
df = df[['Name', 'Age','Nationality','Overall',
       'Potential', 'Club','Value', 'Wage', 'Special','International Reputation','Skill Moves', 'Work Rate','Height', 'Weight','Best Position', 'Best Overall Rating']]
df.head()

,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,International Reputation,Skill Moves,Work Rate,Height,Weight,Best Position,Best Overall Rating
0,Bruno Fernandes,26,Portugal,88,89,Manchester United,€107.5M,€250K,2341,3.0,4.0,High/ High,179cm,69kg,CAM,88.0
1,L. Goretzka,26,Germany,87,88,FC Bayern München,€93M,€140K,2314,4.0,3.0,High/ Medium,189cm,82kg,CM,87.0
2,L. Suárez,34,Uruguay,88,88,Atlético de Madrid,€44.5M,€135K,2307,5.0,3.0,High/ Medium,182cm,83kg,ST,88.0
3,K. De Bruyne,30,Belgium,91,91,Manchester City,€125.5M,€350K,2304,4.0,4.0,High/ High,181cm,70kg,CM,91.0
4,M. Acuña,29,Argentina,84,84,Sevilla FC,€37M,€45K,2292,2.0,4.0,High/ High,172cm,69kg,LB,84.0


In [ ]:
for column in ('Value', 'Wage'):
  for index in range(0, len(df[column])):
    value = str(df[column].loc[index])

    value = value.replace('€','')

    if '.' in value:
      value = value.replace('.', '')
    if 'K' in value:
      value = value.replace('K', '000')
    if 'M' in value:
      value = value.replace('M', '000000')

    df[column].loc[index] = int(value)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
df['Value']= pd.to_numeric(df['Value'],errors = 'coerce')
df['Wage']= pd.to_numeric(df['Wage'],errors = 'coerce')

In [ ]:
#se agrega una nueva columna donde se encasilla a los jugadores en posiciones generales, siendo estas Arquero, Defensor, Volante y Delantero

map_position={'CAM':'Volante','CM':'Volante', 'LB':'Defensor',  'ST':'Delantero', 'CDM':'Volante', 'CB':'Defensor', 'RB':'Defensor', 'LM':'Volante', 
              'RW':'Volante', 'LW':'Volante', 'CF':'Delantero', 'LWB':'Defensor', 'RM':'Volante','RWB':'Defensor', 'GK':'Arquero'}

df['General Position'] = df['Best Position'].map(map_position)

df.head()

,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,International Reputation,Skill Moves,Work Rate,Height,Weight,Best Position,Best Overall Rating,General Position
0,Bruno Fernandes,26,Portugal,88,89,Manchester United,1075000000,250000,2341,3.0,4.0,High/ High,179cm,69kg,CAM,88.0,Volante
1,L. Goretzka,26,Germany,87,88,FC Bayern München,93000000,140000,2314,4.0,3.0,High/ Medium,189cm,82kg,CM,87.0,Volante
2,L. Suárez,34,Uruguay,88,88,Atlético de Madrid,445000000,135000,2307,5.0,3.0,High/ Medium,182cm,83kg,ST,88.0,Delantero
3,K. De Bruyne,30,Belgium,91,91,Manchester City,1255000000,350000,2304,4.0,4.0,High/ High,181cm,70kg,CM,91.0,Volante
4,M. Acuña,29,Argentina,84,84,Sevilla FC,37000000,45000,2292,2.0,4.0,High/ High,172cm,69kg,LB,84.0,Defensor


In [ ]:
# Analicemos brevemente el dataset
df.shape

(16710, 17)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16710 entries, 0 to 16709
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Name                      16710 non-null  object 
 1   Age                       16710 non-null  int64  
 2   Nationality               16710 non-null  object 
 3   Overall                   16710 non-null  int64  
 4   Potential                 16710 non-null  int64  
 5   Club                      16446 non-null  object 
 6   Value                     16710 non-null  int64  
 7   Wage                      16710 non-null  int64  
 8   Special                   16710 non-null  int64  
 9   International Reputation  16710 non-null  float64
 10  Skill Moves               16710 non-null  float64
 11  Work Rate                 16710 non-null  object 
 12  Height                    16710 non-null  object 
 13  Weight                    16710 non-null  object 
 14  Best P

**Insights**:

* El dataset se compone de 17 columnas de las cuales 9 corresponden a variables independientes, que usaremos para predecir el target.

* International Reputation es la variable a predecir. Todas las variables son numéricas, ya sea enteras o reales y no tiene valores nulos.

In [ ]:
#Separamos en X e y
X = df.drop(["International Reputation"], axis=1)
y = df["International Reputation"]
X.drop(["Name",  "Nationality", "Club", 'Work Rate', "Height", "Weight",'Best Position', "General Position"], axis=1, inplace=True)

In [ ]:
#Separamos en train y test!
(X_train, X_test, y_train, y_test) = train_test_split(X, y, stratify=y, test_size=0.30, random_state=11)

In [ ]:
#Creamos un arbol de decisión sencillo y lo fiteamos
tree = DecisionTreeClassifier(random_state=11)
tree.fit(X_train, y_train)

DecisionTreeClassifier(random_state=11)

In [ ]:
y_test_pred = tree.predict(X_test) #Prediccion en Test

In [ ]:
from sklearn.metrics import accuracy_score

#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print('% de aciertos sobre el set de evaluación:',test_accuracy)

% de aciertos sobre el set de evaluación: 0.8661480151605825


In [ ]:
#Creamos un random forest!
model = RandomForestClassifier(random_state=11, n_estimators=200,
                               class_weight="balanced", max_features="log2")
model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_features='log2',
                       n_estimators=200, random_state=11)

In [ ]:
y_test_pred = model.predict(X_test) #Prediccion en Test

In [ ]:
#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print('% de aciertos sobre el set de evaluación:',test_accuracy)

% de aciertos sobre el set de evaluación: 0.8904847396768402


Conclusiones!

Simplemente utilizando el algoritmo de random forest, obtenemos un mejor accuracy que haciendo uso de un DecisionTreeClassifier 😉